<a href="https://colab.research.google.com/github/jaredlan1/getting_started_with_ML/blob/main/Copy_of_HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
class VanillaNetwork(nn.Module):
  def __init__(self):
    super().__init__()

    self.network = nn.Sequential(
        nn.Linear(13.24),
        nn.ReLU(),
        nn.Linear(24,36),
        nn.ReLU(),
        nn.Linear(36,1)
    )
    def forward(self, x):
       self.network(x)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):

    model.train()

    for X, y in dataloader:

      # Compute prediction error
      pred = model(X)
      loss = loss_fn(pred, y)

        # Backpropagation
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

      train_loss += loss.item()
    train_loss /= len(dataloader)
    print(f"Train Error: \n Avg loss: {train_loss:>8f} \n")

In [ ]:
def test(dataloader, model, loss_fn):

    num_batches = len(dataloader)
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for X, y in dataloader:

            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches

    print(f"Test loss: {test_loss:>8f} \n")

In [ ]:
def loaddata():


In [ ]:
model = VanillaNetwork()
loss_fn = nn.MSELoss
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

TypeError: Linear.__init__() missing 1 required positional argument: 'out_features'

In [ ]:
epochs = 1000
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loss = train(train_dataloader, model, loss_fn, optimizer)
    test_loss = test(test_dataloader, model, loss_fn)
    train_losses.append(train_loss)
    test_losses.append(test_loss)
plt.figure()
plt.plot(range(epochs), train_losses, label='train')
plt.plot(range(epochs), test_losses, label='test')
plt.show()



# Task
Prepare the 'auto-mpg.data' dataset for machine learning by using 'auto-mpg.names' to infer column information, loading the data, handling missing values, encoding categorical features, and splitting the data into training and test sets.

## Load and Parse auto-mpg.names

### Subtask:
Read the `auto-mpg.names` file to understand the column structure, identify feature names, and determine the target variable ('mpg').


**Reasoning**:
The first instruction is to download the `auto-mpg.data` and `auto-mpg.names` files. I will use `!wget` command to download these files from the UCI Machine Learning Repository.



# Task
Prepare the 'auto-mpg.data' dataset for machine learning by loading it from `drive/MyDrive/auto-mpg.data`, using `drive/MyDrive/auto-mpg.names` for column information, handling missing values, encoding categorical features, and splitting the data into training and test sets.

## Load and Parse auto-mpg.names from Drive

### Subtask:
Read the `auto-mpg.names` file from your `drive/MyDrive` to understand the column structure, identify feature names, and determine the target variable ('mpg').


**Reasoning**:
The first instruction for the subtask is to mount Google Drive to access files from `drive/MyDrive`.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')
print("Google Drive mounted successfully.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted successfully.


**Reasoning**:
Now that Google Drive is mounted, I will open and read the `auto-mpg.names` file from `drive/MyDrive` to understand its content, which describes the column structure and identifies the target variable.



In [ ]:
import os

directory_path = '/content/drive/MyDrive/Physics Informed ML/HW/auto_mpg'

try:
    # List contents of the directory
    contents = os.listdir(directory_path)
    print(f"Contents of '{directory_path}':")
    for item in contents:
        print(item)
except FileNotFoundError:
    print(f"Error: The directory '{directory_path}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

Contents of '/content/drive/MyDrive/Physics Informed ML/HW/auto_mpg':
auto-mpg.data
auto-mpg.data-original
auto-mpg.names
Index
.ipynb_checkpoints


In [ ]:
names_file_path = '/content/drive/MyDrive/Physics Informed ML/HW/auto_mpg/auto-mpg.names'

try:
    with open(names_file_path, 'r') as f:
        names_content = f.read()
    print(names_content)
except FileNotFoundError:
    print(f"Error: The file '{names_file_path}' was not found. Please ensure the path is correct.")
except Exception as e:
    print(f"An error occurred: {e}")

1. Title: Auto-Mpg Data

2. Sources:
   (a) Origin:  This dataset was taken from the StatLib library which is
                maintained at Carnegie Mellon University. The dataset was 
                used in the 1983 American Statistical Association Exposition.
   (c) Date: July 7, 1993

3. Past Usage:
    -  See 2b (above)
    -  Quinlan,R. (1993). Combining Instance-Based and Model-Based Learning.
       In Proceedings on the Tenth International Conference of Machine 
       Learning, 236-243, University of Massachusetts, Amherst. Morgan
       Kaufmann.

4. Relevant Information:

   This dataset is a slightly modified version of the dataset provided in
   the StatLib library.  In line with the use by Ross Quinlan (1993) in
   predicting the attribute "mpg", 8 of the original instances were removed 
   because they had unknown values for the "mpg" attribute.  The original 
   dataset is available in the file "auto-mpg.data-original".

   "The data concerns city-cycle fuel consumptio

## Load `auto-mpg.data` into DataFrame from Drive

### Subtask:
Load the `auto-mpg.data` file from the corresponding path in your `drive/MyDrive` into a pandas DataFrame, using the column names extracted from `auto-mpg.names` and excluding the 'car name' column.


**Reasoning**:
The first instruction is to import the pandas library, which is necessary for creating and manipulating DataFrames.



In [ ]:
import pandas as pd
print("Pandas library imported as pd.")

Pandas library imported as pd.


**Reasoning**:
Now that the pandas library is imported, I will proceed to define the column names, construct the file path, load the `auto-mpg.data` into a pandas DataFrame, handling missing values, and then display the head and information of the DataFrame to verify the loading process and data types.



In [ ]:
column_names = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model year', 'origin', 'car name']
data_file_path = os.path.join(directory_path, 'auto-mpg.data')

# Load the data, handling '?' as missing values and specifying whitespace as separator
df_auto_mpg = pd.read_csv(data_file_path, names=column_names, na_values='?', delim_whitespace=True)

# Drop the 'car name' column as it's not needed for analysis
df_auto_mpg = df_auto_mpg.drop('car name', axis=1)

print("First 5 rows of the DataFrame:")
print(df_auto_mpg.head())

print("\nDataFrame Information:")
df_auto_mpg.info()

First 5 rows of the DataFrame:
    mpg  cylinders  displacement  horsepower  weight  acceleration  \
0  18.0          8         307.0       130.0  3504.0          12.0   
1  15.0          8         350.0       165.0  3693.0          11.5   
2  18.0          8         318.0       150.0  3436.0          11.0   
3  16.0          8         304.0       150.0  3433.0          12.0   
4  17.0          8         302.0       140.0  3449.0          10.5   

   model year  origin  
0          70       1  
1          70       1  
2          70       1  
3          70       1  
4          70       1  

DataFrame Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     398 non-null    int64  
 2   displacement  398 non-null    float64
 3   horsepower    392 non-null    float64
 4   weight       

/tmp/ipython-input-118887064.py:5: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_auto_mpg = pd.read_csv(data_file_path, names=column_names, na_values='?', delim_whitespace=True)


**Reasoning**:
The previous code produced a `FutureWarning` because `delim_whitespace=True` is deprecated. I will update the `pd.read_csv` function call to use `sep='\s+'` as suggested by the warning for better practice.



In [ ]:
column_names = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model year', 'origin', 'car name']
data_file_path = os.path.join(directory_path, 'auto-mpg.data')

# Load the data, handling '?' as missing values and specifying whitespace as separator
df_auto_mpg = pd.read_csv(data_file_path, names=column_names, na_values='?', sep='\s+')

# Drop the 'car name' column as it's not needed for analysis
df_auto_mpg = df_auto_mpg.drop('car name', axis=1)

print("First 5 rows of the DataFrame:")
print(df_auto_mpg.head())

print("\nDataFrame Information:")
df_auto_mpg.info()

First 5 rows of the DataFrame:
    mpg  cylinders  displacement  horsepower  weight  acceleration  \
0  18.0          8         307.0       130.0  3504.0          12.0   
1  15.0          8         350.0       165.0  3693.0          11.5   
2  18.0          8         318.0       150.0  3436.0          11.0   
3  16.0          8         304.0       150.0  3433.0          12.0   
4  17.0          8         302.0       140.0  3449.0          10.5   

   model year  origin  
0          70       1  
1          70       1  
2          70       1  
3          70       1  
4          70       1  

DataFrame Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     398 non-null    int64  
 2   displacement  398 non-null    float64
 3   horsepower    392 non-null    float64
 4   weight       

<>:5: SyntaxWarning: invalid escape sequence '\s'
<>:5: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipython-input-2403202165.py:5: SyntaxWarning: invalid escape sequence '\s'
  df_auto_mpg = pd.read_csv(data_file_path, names=column_names, na_values='?', sep='\s+')


**Reasoning**:
The previous code generated a `SyntaxWarning` because `\s` was interpreted as an escape sequence rather than a raw string for the regular expression separator. I will fix this by using a raw string literal `r'\s+'` for the `sep` argument in `pd.read_csv`.



In [ ]:
column_names = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model year', 'origin', 'car name']
data_file_path = os.path.join(directory_path, 'auto-mpg.data')

# Load the data, handling '?' as missing values and specifying whitespace as separator
df_auto_mpg = pd.read_csv(data_file_path, names=column_names, na_values='?', sep=r'\s+')

# Drop the 'car name' column as it's not needed for analysis
df_auto_mpg = df_auto_mpg.drop('car name', axis=1)

print("First 5 rows of the DataFrame:")
print(df_auto_mpg.head())

print("\nDataFrame Information:")
df_auto_mpg.info()

First 5 rows of the DataFrame:
    mpg  cylinders  displacement  horsepower  weight  acceleration  \
0  18.0          8         307.0       130.0  3504.0          12.0   
1  15.0          8         350.0       165.0  3693.0          11.5   
2  18.0          8         318.0       150.0  3436.0          11.0   
3  16.0          8         304.0       150.0  3433.0          12.0   
4  17.0          8         302.0       140.0  3449.0          10.5   

   model year  origin  
0          70       1  
1          70       1  
2          70       1  
3          70       1  
4          70       1  

DataFrame Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     398 non-null    int64  
 2   displacement  398 non-null    float64
 3   horsepower    392 non-null    float64
 4   weight       

## Handle Missing Values

### Subtask:
Inspect the loaded DataFrame for missing values and remove any rows that contain them.


**Reasoning**:
To address the subtask of handling missing values, I will first inspect the `df_auto_mpg` DataFrame to identify columns with missing values and their counts, then remove rows containing any missing values, and finally verify that all missing values have been successfully removed.



In [ ]:
print("Missing values before dropping rows:")
print(df_auto_mpg.isnull().sum())

# Remove rows with any missing values
df_auto_mpg = df_auto_mpg.dropna()

print("\nMissing values after dropping rows:")
print(df_auto_mpg.isnull().sum())

print("\nShape of the DataFrame after dropping rows:", df_auto_mpg.shape)

Missing values before dropping rows:
mpg             0
cylinders       0
displacement    0
horsepower      6
weight          0
acceleration    0
model year      0
origin          0
dtype: int64

Missing values after dropping rows:
mpg             0
cylinders       0
displacement    0
horsepower      0
weight          0
acceleration    0
model year      0
origin          0
dtype: int64

Shape of the DataFrame after dropping rows: (392, 8)


## Separate Features and Target

### Subtask:
Split the cleaned DataFrame into features (X) and the target variable (y, which is 'mpg').


**Reasoning**:
To separate the features and the target variable, I will create a new DataFrame `X` by dropping the 'mpg' column from `df_auto_mpg` and a Series `y` containing the 'mpg' column. Finally, I will print the first 5 rows of both `X` and `y` to confirm the separation.



In [ ]:
X = df_auto_mpg.drop('mpg', axis=1)
y = df_auto_mpg['mpg']

print("First 5 rows of features (X):")
print(X.head())

print("\nFirst 5 rows of target (y):")
print(y.head())

First 5 rows of features (X):
   cylinders  displacement  horsepower  weight  acceleration  model year  \
0          8         307.0       130.0  3504.0          12.0          70   
1          8         350.0       165.0  3693.0          11.5          70   
2          8         318.0       150.0  3436.0          11.0          70   
3          8         304.0       150.0  3433.0          12.0          70   
4          8         302.0       140.0  3449.0          10.5          70   

   origin  
0       1  
1       1  
2       1  
3       1  
4       1  

First 5 rows of target (y):
0    18.0
1    15.0
2    18.0
3    16.0
4    17.0
Name: mpg, dtype: float64


## Encode Categorical Features

### Subtask:
Identify and one-hot encode any categorical features, specifically the 'origin', 'cylinders', and 'model year' columns, to prepare the data for model training.


### Identify Categorical Columns

**Reasoning**:
The categorical columns to be one-hot encoded are explicitly identified as 'cylinders', 'model year', and 'origin' based on the problem description and the subtask instructions. These will be processed using `pd.get_dummies()` in the next step to convert them into a numerical format suitable for machine learning models.

**Reasoning**:
Now that the categorical columns have been identified, I will use `pd.get_dummies()` to perform one-hot encoding on 'cylinders', 'model year', and 'origin', setting `drop_first=True` to prevent multicollinearity. Then, I will update the `X` DataFrame with the encoded features and display its head and shape to verify the transformation.

